# Tugas 5 (LSA Topic Modelling)

## Read Data

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

Data didapatkan dari proses scrapping data youtube


proses scrapping dapat dilihat pada halaman berikut [scrapping_comment_yt](https://hamedayani114.github.io/prosaindata/scrapping_comment_yt.html)

In [2]:
import pandas as pd
df=pd.read_csv('https://raw.githubusercontent.com/triasmidwi/dataset/main/commentyoutube.csv')
df

,comment,comment (clean)
0,ID\nSkip navigation\nSign in\nWorship Piano: B...,id skip navigation sign ini worship piano begi...
1,"Pak Prabowo Subianto saya mohon,\nAnda jangan ...",pak prabowo subianto saya mohon anda jangan be...
2,gw dan sekeluarga adalah loyalis PDI Perjuanga...,gue dan sekeluarga adalah loyalis pdi perjuang...
3,"ganjar salah satu yang menolak israel, berakib...",ganjar salah satu yang menolak israel berakiba...
4,"Pak Prabowo sekarang auranya adem,beda waktu n...",pak prabowo sekarang auranya adem beda waktu n...
...,...,...
1160,Di,di
1161,Siapa pula yg mau pilih prabowo....history mas...,siapa pula yang mau pilih prabowo history masa...
1162,Lembaga survei bayaran\nAslinya pak Anies yg t...,lembaga survei bayaran aslinya pak anies yang ...
1163,Siapapun calonya presiden 2024\nKami rakyat t...,siapapun calonya presiden kami rakyat tetap an...


## Modelling

In [3]:
import nltk
nltk.download('stopwords', quiet=True)

True

In [4]:
from nltk.tokenize import RegexpTokenizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords
stopwords = stopwords.words('indonesian')

# Membentuk matriks dokumen x kata
tokenizer = RegexpTokenizer(r'\w+')
vectorizer = TfidfVectorizer(lowercase=True,
                        stop_words=stopwords,
                        tokenizer = tokenizer.tokenize)

tfidf_matrix = vectorizer.fit_transform(df['comment (clean)'])

# Melakukan dekomposisi matriks dengan SVD
svd_model = TruncatedSVD(n_components=4)
lsa_matrix = svd_model.fit_transform(tfidf_matrix)

### bobot kata terhadap masing masing topik

In [5]:
# bobot kata terhadap masing masing topik
terms = vectorizer.get_feature_names_out()

for index, component in enumerate(svd_model.components_):
    zipped = zip(terms, component)
    top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:2]
    print("Topic "+str(index)+": ",top_terms_key)

Topic 0:  [('prabowo', 0.7521371227420962), ('ganjar', 0.33719092187096467)]
Topic 1:  [('ganjar', 0.7501242467068866), ('pilih', 0.18735525512545587)]
Topic 2:  [('mahfud', 0.5855374668804467), ('md', 0.5182123784120537)]
Topic 3:  [('anis', 0.5691506442005095), ('pilih', 0.3854966845116057)]


### bobot setiap topik terhadap dokumen

In [6]:
# bobot setiap topik terhadap  dokumen
df_lsa = pd.DataFrame(lsa_matrix, columns=["Topik 0", "Topik 1", "Topik 2", "Topik 3"])
df_lsa = pd.concat([df["comment (clean)"], df_lsa], axis=1)
df_lsa['Topik']= df_lsa[['Topik 0', 'Topik 1', 'Topik 2', 'Topik 3']].apply(lambda x: x.argmax(), axis=1)

df_lsa

,comment (clean),Topik 0,Topik 1,Topik 2,Topik 3,Topik
0,id skip navigation sign ini worship piano begi...,0.152409,0.051503,-0.010322,0.048414,0
1,pak prabowo subianto saya mohon anda jangan be...,0.307130,0.188796,0.026529,-0.117650,0
2,gue dan sekeluarga adalah loyalis pdi perjuang...,0.054242,0.070589,-0.005598,0.014279,1
3,ganjar salah satu yang menolak israel berakiba...,0.071629,0.146154,0.047684,-0.041205,1
4,pak prabowo sekarang auranya adem beda waktu n...,0.098525,-0.021332,-0.032976,-0.032009,0
...,...,...,...,...,...,...
1160,di,0.000000,0.000000,0.000000,0.000000,0
1161,siapa pula yang mau pilih prabowo history masa...,0.107986,0.008436,-0.050965,0.058128,0
1162,lembaga survei bayaran aslinya pak anies yang ...,0.030167,0.034284,-0.034703,0.114658,3
1163,siapapun calonya presiden kami rakyat tetap an...,0.118899,0.049287,-0.130413,0.277299,3


In [7]:
df_lsa['Topik'].value_counts()


0    563
3    325
1    217
2     60
Name: Topik, dtype: int64